In [7]:

!apt update
!apt install nmap -y
!pip install python-nmap

import nmap

# Initialize Nmap PortScanner
nm = nmap.PortScanner()

Reading package lists... Done
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


PortScannerError: 'nmap program was not found in path. PATH is : /opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin'

## Nmap Host Discovery (`-sn`)

The `-sn` option in Nmap performs a **ping scan** (host discovery) without scanning ports. It is used to identify which hosts are **up** on a network without probing for open ports.

```python
nm.scan(hosts='10.100.22.0/24', arguments='-sn')
print(nm.all_hosts())

In [9]:
# 安装nmap（适用于Colab/Kaggle等基于Debian的环境）
!apt update
!apt install nmap -y

# 验证安装
!nmap --version

# 如果已安装python-nmap，重新导入
import importlib
import nmap
importlib.reload(nmap)

# 现在测试
nm = nmap.PortScanner()
print("✅ nmap PortScanner初始化成功!")

Reading package lists... Done
E: List directory /var/lib/apt/lists/partial is missing. - Acquire (13: Permission denied)
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
/bin/bash: line 1: nmap: command not found


PortScannerError: 'nmap program was not found in path. PATH is : /opt/conda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin'

In [31]:
# Host discovery (ping scan)
nm.scan(hosts='10.100.22.0/24', arguments='-sn')
print(nm.all_hosts())

['10.100.22.101', '10.100.22.110', '10.100.22.115', '10.100.22.117', '10.100.22.118', '10.100.22.12', '10.100.22.121', '10.100.22.122', '10.100.22.123', '10.100.22.124', '10.100.22.127', '10.100.22.131', '10.100.22.135', '10.100.22.144', '10.100.22.176', '10.100.22.19', '10.100.22.20', '10.100.22.209', '10.100.22.225', '10.100.22.230', '10.100.22.253', '10.100.22.254', '10.100.22.28', '10.100.22.31', '10.100.22.45', '10.100.22.6', '10.100.22.68']


In [26]:
target_ip = '10.100.22.127'

## SYN Scan for Open Ports (`-sS`)

The **SYN scan** (`-sS`) is one of the most commonly used Nmap scanning techniques. It is also called a **half-open scan** because it doesn’t complete the full TCP handshake. Instead, it sends a **SYN packet** to the target and waits for a response. If the port is open, the target will respond with a **SYN-ACK**, indicating it is ready to establish a connection. Nmap then immediately sends an **RST** (reset) packet, closing the connection before it is fully established.

```python
nm.scan(target_ip, arguments='-sS')


In [ ]:
# SYN scan for open ports
nm.scan(target_ip, arguments='-sS')

In [45]:
for host in nm.all_hosts():
    print('----------------------------------------------------')
    print(f'Host : {host} {nm[host].hostname()}')
    print(f'State : {nm[host].state()}')
    for proto in nm[host].all_protocols():
        print('----------')
        print(f'Protocol : {proto}')
        lport = nm[host][proto].keys()
        for port in lport:
            print (f'port : {port}\tstate : {nm[host][proto][port]["state"]}')

----------------------------------------------------
Host : 10.100.22.127 
State : up
----------
Protocol : tcp
port : 135	state : open
port : 139	state : open
port : 445	state : open
port : 1521	state : open
port : 8080	state : open


In [46]:
print(f"Scan results for {target_ip}:\n")
for protocol in nm[target_ip].all_protocols():
    print(f"Protocol: {protocol}")
    ports = nm[target_ip][protocol].keys()
    for port in ports:
        print(f"Port: {port}\tState: {nm[target_ip][protocol][port]['state']}")
        print(f"Service: {nm[target_ip][protocol][port]['name']}")
        print(f"Version: {nm[target_ip][protocol][port].get('version', 'Unknown')}")

Scan results for 10.100.22.127:

Protocol: tcp
Port: 135	State: open
Service: msrpc
Version: 
Port: 139	State: open
Service: netbios-ssn
Version: 
Port: 445	State: open
Service: microsoft-ds
Version: 
Port: 1521	State: open
Service: oracle
Version: 
Port: 8080	State: open
Service: http-proxy
Version: 


## Service Version Detection (`-sV`)

The **Service Version Detection** scan (`-sV`) in Nmap is used to determine the version of the services running on the open ports. This scan goes beyond simply identifying open ports—it attempts to determine the software version running behind each port by probing and analyzing the responses from the services.

```python
nm.scan(target_ip, arguments='-sV')
nm[target_ip]['tcp']


In [53]:
# Service version detection
nm.scan(target_ip, arguments='-sV')
nm[target_ip]['tcp']

{135: {'state': 'open',
  'reason': 'syn-ack',
  'name': 'msrpc',
  'product': 'Microsoft Windows RPC',
  'version': '',
  'extrainfo': '',
  'conf': '10',
  'cpe': 'cpe:/o:microsoft:windows'},
 139: {'state': 'open',
  'reason': 'syn-ack',
  'name': 'netbios-ssn',
  'product': 'Microsoft Windows netbios-ssn',
  'version': '',
  'extrainfo': '',
  'conf': '10',
  'cpe': 'cpe:/o:microsoft:windows'},
 445: {'state': 'open',
  'reason': 'syn-ack',
  'name': 'microsoft-ds',
  'product': '',
  'version': '',
  'extrainfo': '',
  'conf': '3',
  'cpe': ''},
 1521: {'state': 'open',
  'reason': 'syn-ack',
  'name': 'oracle-tns',
  'product': 'Oracle TNS listener',
  'version': '11.2.0.2.0',
  'extrainfo': 'unauthorized',
  'conf': '10',
  'cpe': 'cpe:/a:oracle:database_server:11.2.0.2.0'},
 8080: {'state': 'open',
  'reason': 'syn-ack',
  'name': 'http',
  'product': 'Oracle XML DB Enterprise Edition httpd',
  'version': '',
  'extrainfo': '',
  'conf': '10',
  'cpe': 'cpe:/a:oracle:database_s

In [10]:
# OS detection
nm.scan(target_ip, arguments='-O')
print(nm[target_ip]['osmatch'])

[{'name': 'Microsoft Windows 10 1709 - 21H2', 'accuracy': '100', 'line': '74990', 'osclass': [{'type': 'general purpose', 'vendor': 'Microsoft', 'osfamily': 'Windows', 'osgen': '10', 'accuracy': '100', 'cpe': ['cpe:/o:microsoft:windows_10']}]}]


## Running an NSE Script to Check for Vulnerabilities

Nmap’s **Nmap Scripting Engine (NSE)** allows users to extend Nmap's functionality by running custom scripts. These scripts can perform tasks like vulnerability detection, network discovery, and service version detection. The following example shows how to run the `oracle-tns-version` NSE script to detect the version of an Oracle TNS Listener on port 1521.

```python
nm.scan(target_ip, arguments='--script oracle-tns-version -p 1521')
```

### Other NSE Script Options

- **`http-vuln-cve2015-1635`**: 
  - Checks for the CVE-2015-1635 vulnerability in Microsoft HTTP.sys, which can lead to remote code execution.
  
- **`ftp-anon`**: 
  - Detects whether an FTP server allows anonymous login, which could be a security risk if sensitive files are accessible.
  
- **`smb-vuln-ms17-010`**: 
  - Checks for the vulnerability associated with **EternalBlue** (CVE-2017-0144), which was used in the WannaCry ransomware attack.
  
- **`ssl-heartbleed`**: 
  - Checks if a system is vulnerable to the **Heartbleed** bug (CVE-2014-0160) in OpenSSL, which allows an attacker to extract sensitive information from the server.

- **`oracle-sid-brute`**: 
  - Attempts to brute-force Oracle System Identifier (SID) names to find the database name, which can help attackers gain access to the database.

- **`dns-brute`**: 
  - Performs a DNS brute-force attack to enumerate subdomains.
  
- **`mysql-info`**: 
  - Gathers MySQL database information, including version and supported protocols.



In [57]:
# Running an NSE script to check for vulnerabilities
nm.scan(target_ip, arguments='--script oracle-tns-version -p 1521')

{'nmap': {'command_line': 'nmap -oX - --script oracle-tns-version -p 1521 10.100.22.127',
  'scaninfo': {'tcp': {'method': 'syn', 'services': '1521'}},
  'scanstats': {'timestr': 'Tue Oct 22 17:25:25 2024',
   'elapsed': '0.20',
   'uphosts': '1',
   'downhosts': '0',
   'totalhosts': '1'}},
 'scan': {'10.100.22.127': {'hostnames': [{'name': '', 'type': ''}],
   'addresses': {'ipv4': '10.100.22.127', 'mac': '50:9A:4C:1D:11:44'},
   'vendor': {'50:9A:4C:1D:11:44': 'Dell'},
   'status': {'state': 'up', 'reason': 'arp-response'},
   'tcp': {1521: {'state': 'open',
     'reason': 'syn-ack',
     'name': 'oracle',
     'product': 'Oracle TNS listener',
     'version': '11.2.0.2.0',
     'extrainfo': 'unauthorized',
     'conf': '10',
     'cpe': 'cpe:/a:oracle:database_server:11.2.0.2.0'}}}}}

## Nmap Evasion Techniques

### 1. **Decoy Scan** (`-D RND:10`)
The **decoy scan** (`-D`) uses multiple IP addresses as decoys to hide the real IP of the attacker. The option `RND:10` generates 10 random decoy IP addresses. This method makes it harder for security systems or administrators to detect the actual source of the scan by mixing legitimate packets with packets from decoy IPs.

```python
nm.scan(target_ip, arguments='-D RND:10')
```

Use case: Avoid detection by firewalls, intrusion detection systems (IDS), or administrators who are monitoring for suspicious activity.

In [58]:
# Decoy scan
nm.scan(target_ip, arguments='-D RND:10')
print(nm.command_line())

nmap -oX - -D RND:10 10.100.22.127


### 2. Fragmented Packet Scan (`-f`)
A **fragmented packet scan** (`-f`) breaks the scanning packets into smaller fragments, making it more difficult for some firewalls or IDS/IPS systems to detect the scan. This technique is useful for evasion, as many security systems may not reassemble the fragmented packets for inspection.

```python
nm.scan(target_ip, arguments='-f')
```
Use case: Bypass basic firewalls or intrusion detection systems that fail to properly inspect fragmented packets.

In [ ]:
# Fragmented packet scan
nm.scan(target_ip, arguments='-f')
# print(nm.command_line())

### 3. Timing Adjustment (`-T4`)
The **timing adjustment option** (`-T4`) configures the scan to use aggressive timing, which makes the scan faster. This setting increases the speed of the scan by reducing delays between packets and probes. T4 is suitable for networks where speed is a priority and stealth is less of a concern.

```python
nm.scan(target_ip, arguments='-T4')
```
Use case: Perform fast scans in internal networks where speed is important and stealth is not required.

| Timing Option | Name        | Speed       | Stealth   | Use Case                                                 |
|---------------|-------------|-------------|-----------|----------------------------------------------------------|
| `-T0`         | Paranoid    | Very Slow   | Very High | High-security environments; avoiding detection is critical. |
| `-T1`         | Sneaky      | Slow        | High      | Lightly monitored networks; stealth is important.         |
| `-T2`         | Polite      | Moderate    | Medium    | Production environments; minimize disruption.             |
| `-T3`         | Normal      | Balanced    | Balanced  | Default option; general-purpose scanning.                 |
| `-T4`         | Aggressive  | Fast        | Low       | Internal networks; speed is more important than stealth.  |
| `-T5`         | Insane      | Very Fast   | Very Low  | Lab environments; prioritize speed over stealth.          |


In [ ]:
# Adjusting timing
nm.scan(target_ip, arguments='-T4')
# print(nm.command_line())